In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


***
***

# **ETL:**


***
***

##  ***1_*** **User Reviews:**






## importar Librerias:




In [2]:
# Importar librerias necesarias
import pandas as pd
import ast
import pyarrow as py
from pandas import json_normalize


***
***

### Cargar JSON y guardar en una variable






In [2]:
# Generar un Dataframe para guardar los datos provenientes del archivo Json anidado
data_list = []
# Se realiza una iteracion de las lineas del archivo por cada linea
file = '/content/drive/MyDrive/Colab Notebooks/PI_1/Data/australian_user_reviews.json'
with open(file, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue
# Guardar el archivo en una variable y como un DataFrame
user_reviews = pd.DataFrame(data_list)

In [3]:
# Crea un df con el archivo review y accede a la columna que interesa desanidar.
df_reviews_Open = user_reviews.explode('reviews')

In [4]:
# Creo un nuevo DataFrame donde esta la info desanidada en columnas y se concatena con el DataFrame explotado
df_user_reviews = pd.concat([df_reviews_Open.drop(['reviews'], axis=1), df_reviews_Open['reviews'].apply(pd.Series)], axis=1)

In [5]:
# Observar las caracteristicas generales, como de sus variables
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59333 entries, 0 to 25798
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      59333 non-null  object 
 1   user_url     59333 non-null  object 
 2   funny        59305 non-null  object 
 3   posted       59305 non-null  object 
 4   last_edited  59305 non-null  object 
 5   item_id      59305 non-null  object 
 6   helpful      59305 non-null  object 
 7   recommend    59305 non-null  object 
 8   review       59305 non-null  object 
 9   0            0 non-null      float64
dtypes: float64(1), object(9)
memory usage: 5.0+ MB


In [6]:
# Informacion sobre la estructura del DataFrame
df_user_reviews.shape

(59333, 10)



 *   El Dataframe df_user_reviews posee 10 columnas y 59333 filas, la ultima columna posee todos los datos nulos y el indice no esta bien indexado




In [7]:
# Visualizacion de las 5 primeras filas
df_user_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN


In [8]:
# Reindexar las filas, debido a que quedaron desorganizadas luego de concatenar los DataFrame
df_user_reviews = df_user_reviews.reset_index(drop=True)

In [9]:
# Eliminar columnas 0 que posee todos los datos nulos
df_user_reviews = df_user_reviews.drop([0], axis=1)

In [10]:
# Verificar si cada fila tiene todos los valores nulos
filas_con_nulos = df_user_reviews.isna().all(axis=1).sum()
# Mostrar las filas que tienen todos los valores nulos
print(f'Cantidad de Filas con todos Valores Nulos: {filas_con_nulos}')


Cantidad de Filas con todos Valores Nulos: 0


In [11]:
# Modificar los nombres de las columnas
df_user_reviews = df_user_reviews.rename(columns={
    'user_id': 'Id_Usuario',
    'user_url': 'Url_Usuario',
    'funny': 'funny',
    'posted': 'Posteo',
    'last_edited': 'Ultima_Edicion',
    'item_id': 'Id_Item',
    'helpful': 'Util',
    'recommend': 'Recomendado',
    'review': 'Review'
})

In [12]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Id_Usuario      59333 non-null  object
 1   Url_Usuario     59333 non-null  object
 2   funny           59305 non-null  object
 3   Posteo          59305 non-null  object
 4   Ultima_Edicion  59305 non-null  object
 5   Id_Item         59305 non-null  object
 6   Util            59305 non-null  object
 7   Recomendado     59305 non-null  object
 8   Review          59305 non-null  object
dtypes: object(9)
memory usage: 4.1+ MB




---




### Valores Nulos

In [13]:
# Busacar filas con todos los Valores Nulos
filas_completas_nulas = df_user_reviews.isna().all(axis=1).sum()
print(f' Cantidad de Filas con todos Valores Nulos: {filas_completas_nulas}')

 Cantidad de Filas con todos Valores Nulos: 0


In [14]:
# Cantidad de Valores Nulos por columna
df_user_reviews.isna().sum()

Id_Usuario         0
Url_Usuario        0
funny             28
Posteo            28
Ultima_Edicion    28
Id_Item           28
Util              28
Recomendado       28
Review            28
dtype: int64

In [15]:
# Buscar filas con al menos un valor nulo en columnas específicas
filas_nulas = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion', 'Id_Item', 'Util', 'Recomendado', 'Review']].isna().any(axis=1)
df_user_reviews[filas_nulas]

,Id_Usuario,Url_Usuario,funny,Posteo,Ultima_Edicion,Id_Item,Util,Recomendado,Review
137,gdxsd,http://steamcommunity.com/id/gdxsd,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,76561198094224872,http://steamcommunity.com/profiles/76561198094...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2559,76561198021575394,http://steamcommunity.com/profiles/76561198021...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10080,cmuir37,http://steamcommunity.com/id/cmuir37,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13767,Jaysteeny,http://steamcommunity.com/id/Jaysteeny,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15493,ML8989,http://steamcommunity.com/id/ML8989,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19184,76561198079215291,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20223,76561198079342142,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25056,76561198061996985,http://steamcommunity.com/profiles/76561198061...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26257,76561198108286351,http://steamcommunity.com/profiles/76561198108...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Elimina los Valores nulos que no se pueden rellenar
df_user_reviews = df_user_reviews.dropna(axis=0)

Se eliminan las 28 filas que comparten valores nulos, debido que no se pueden rellenar y no aportan informacion para el analisis posterior.

In [17]:
# Se reinicia el indice para que coincidan
df_user_reviews = df_user_reviews.reset_index(drop=True)



---


### Limpieza de caracteres especiales de las columnas






In [18]:
# Nombre de las columnas
df_user_reviews.columns

Index(['Id_Usuario', 'Url_Usuario', 'funny', 'Posteo', 'Ultima_Edicion',
       'Id_Item', 'Util', 'Recomendado', 'Review'],
      dtype='object')

In [23]:
# Todos los strings en minúsculas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.astype(str).str.lower() if x.dtype == "object" else x)

# Reemplaza '-' por ' ' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace('-', ' ') if x.dtype == "object" else x)

# Reemplaza '!' por '' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace('!', '') if x.dtype == "object" else x)

# Reemplaza '¡' por '' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace('¡', '') if x.dtype == "object" else x)

# Reemplaza '?' por '' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace('?', '') if x.dtype == "object" else x)

# Reemplaza '¿' por '' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace('¿', '') if x.dtype == "object" else x)

# Reemplaza '"' por '' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace('"', '') if x.dtype == "object" else x)

# Reemplaza ''' por '' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace("'", '') if x.dtype == "object" else x)

# Reemplaza ',' por '' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace(",", '') if x.dtype == "object" else x)

# Reemplaza '&' por '' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace("&", '') if x.dtype == "object" else x)

# Reemplaza "''" por '' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace("''", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace("_", '') if x.dtype == "object" else x)

# Reemplaza "." por '' en todas las columnas
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace(".", '') if x.dtype == "object" else x)

# Reemplaza "\s+" por ' ' espacios adicionales
df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace(r'\s+', ' ') if x.dtype == "object" else x)



<ipython-input-23-1cf3c24d4de5>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace('?', '') if x.dtype == "object" else x)
<ipython-input-23-1cf3c24d4de5>:38: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']] = df_user_reviews[['funny', 'Posteo', 'Ultima_Edicion','Id_Item', 'Util', 'Recomendado', 'Review']].apply(lambda x: x.str.replace(".", '') if x.dtype == "



---




### Guardar el DataFrame comprimido en Parquet




In [ ]:
# Guardar el DataFrame comprimido en Parquet
df_user_reviews.to_parquet('df_user_reviews.parquet')

***
***

##  ***2_*** **Steam Games:**




### Cargar JSON y guardar en una variable


In [3]:
# Carga del archivo y guardado en una variable para generar el DataFrame
df_steam_games = pd.read_json('/content/drive/MyDrive/Colab Notebooks/PI_1/Data/output_steam_games.json', lines=True)

In [4]:
# estructura general de Dataframe
df_steam_games.shape
print(f'El DataFrame posee una cantidad de Filas: {df_steam_games.shape[0]} y Columnas: {df_steam_games.shape[1]}')

El DataFrame posee una cantidad de Filas: 120445 y Columnas: 13


In [5]:
# Observar las caracteristicas generales del DataFrame, como de sus variables
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB




*   El DataFrame posee 120445 de las cuales se puede observar que muchos de los valores son nulos




In [6]:
# Obserava las 5 primeras filas y la estructura
df_steam_games.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None




---



In [7]:
# Nombre de las columnas
df_steam_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [8]:
# Modifica los nombres de las columnas
df_steam_games = df_steam_games.rename(columns={
    'publisher': 'Empresa_Publicadora',
    'genres': 'Generos',
    'app_name': 'Nombre_Contenido',
    'title': 'Titulo',
    'url': 'Url',
    'release_date': 'Fecha_Lanzamiento',
    'tags': 'Etiquetas',
    'reviews_url': 'Reviews_Contenido',
    'specs': 'Especificaciones',
    'price': 'Precio',
    'early_access': 'Acceso_Temprano',
    'id': 'Id_Item',
    'developer': 'Desarrollador'
    })



---



### Valores Nulos

In [9]:
# Buscar Filas con todos los Valores nulos
filas_nulas = df_steam_games[df_steam_games.isna().all(axis=1)]
print(f'Cantidad de filas con todos los Valores Nulos: {len(filas_nulas)}')

Cantidad de filas con todos los Valores Nulos: 88310


In [10]:
# Eliminar las filas nulas
df_steam_games = df_steam_games.dropna(axis=0, how='all')

Se eliminan todas las filas que tienen el todos los valores nulos, el numero eliminado de filas es de 88310

In [11]:
# Reindexar las filas, debido a que quedaron desorganizadas luego de concatenar los DataFrame
df_steam_games = df_steam_games.reset_index(drop=True)

In [12]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Empresa_Publicadora  24083 non-null  object 
 1   Generos              28852 non-null  object 
 2   Nombre_Contenido     32133 non-null  object 
 3   Titulo               30085 non-null  object 
 4   Url                  32135 non-null  object 
 5   Fecha_Lanzamiento    30068 non-null  object 
 6   Etiquetas            31972 non-null  object 
 7   Reviews_Contenido    32133 non-null  object 
 8   Especificaciones     31465 non-null  object 
 9   Precio               30758 non-null  object 
 10  Acceso_Temprano      32135 non-null  float64
 11  Id_Item              32133 non-null  float64
 12  Desarrollador        28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.2+ MB


In [13]:
# Cantidad de Valores Nulos por Columna
df_steam_games.isna().sum()

Empresa_Publicadora    8052
Generos                3283
Nombre_Contenido          2
Titulo                 2050
Url                       0
Fecha_Lanzamiento      2067
Etiquetas               163
Reviews_Contenido         2
Especificaciones        670
Precio                 1377
Acceso_Temprano           0
Id_Item                   2
Desarrollador          3299
dtype: int64

In [14]:
df_steam_games.head()

,Empresa_Publicadora,Generos,Nombre_Contenido,Titulo,Url,Fecha_Lanzamiento,Etiquetas,Reviews_Contenido,Especificaciones,Precio,Acceso_Temprano,Id_Item,Desarrollador
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
4,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None




---






### Imputar los Valores Nulos

In [15]:
# Rellenar los valores nulos de la columna 'Desarrollador' con los de la columna 'Empresa_Publicadora'
df_steam_games['Desarrollador'] = df_steam_games['Desarrollador'].fillna(df_steam_games['Empresa_Publicadora'])

In [16]:
# Rellenar los valores nulos de la columna 'Generos' con los de la columna 'Etiquetas'
df_steam_games['Generos'] = df_steam_games['Generos'].fillna(df_steam_games['Etiquetas'])

In [17]:
# Rellenar los valores nulos de la columna 'Titulo' con los de la columna 'Nombre_Contenido'
df_steam_games['Titulo'] = df_steam_games['Titulo'].fillna(df_steam_games['Nombre_Contenido'])

In [18]:
# Reemplazar celdas que contienen 'Free', 'Play', 'Install' y 'Third' en la columna 'price' con 0.00
df_steam_games.loc[df_steam_games['Precio'].str.contains('Free|Play|Install|Third', case=False, na=False), 'Precio'] = 0.00

In [19]:
# Eliminar "Starting at $449.00" de los valores en la columna 'Price'
df_steam_games['Precio'] = df_steam_games['Precio'].replace('Starting at $449.00', 449.00)
df_steam_games['Precio'] = pd.to_numeric(df_steam_games['Precio'], errors='coerce')

In [20]:
# Redondear los decimales en 2
df_steam_games['Precio'] = df_steam_games['Precio'].round(2)

In [21]:
# Rellenar los valores nulos de la columna 'Precio' con el valor -1
df_steam_games['Precio'] = df_steam_games['Precio'].fillna(-1)

In [22]:
# Filas sin nombre ni titulo
fila_sin_nombre = df_steam_games[df_steam_games['Titulo'].isna() == True]
fila_sin_nombre

,Empresa_Publicadora,Generos,Nombre_Contenido,Titulo,Url,Fecha_Lanzamiento,Etiquetas,Reviews_Contenido,Especificaciones,Precio,Acceso_Temprano,Id_Item,Desarrollador
74,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
2580,None,"[Action, Indie]",None,None,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",-1.00,0.0,317160.0,None


In [23]:
# Eliminar las filas seleccionadas del DataFrame original
df_steam_games = df_steam_games.dropna(subset=['Titulo'])
df_steam_games = df_steam_games.dropna(subset=['Fecha_Lanzamiento'])

In [24]:
# Se busca el registro de 'Id_Item' para poder verificar los datos de las columnas
df_steam_games[df_steam_games['Id_Item'].isna()]

,Empresa_Publicadora,Generos,Nombre_Contenido,Titulo,Url,Fecha_Lanzamiento,Etiquetas,Reviews_Contenido,Especificaciones,Precio,Acceso_Temprano,Id_Item,Desarrollador
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [25]:
# Busca el 'id' del juego con faltante
df_steam_games[df_steam_games['Titulo'] == 'Batman: Arkham City - Game of the Year Edition']

,Empresa_Publicadora,Generos,Nombre_Contenido,Titulo,Url,Fecha_Lanzamiento,Etiquetas,Reviews_Contenido,Especificaciones,Precio,Acceso_Temprano,Id_Item,Desarrollador
1068,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,200260.0,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [26]:
# Reemplazar el Valor Nulo del id en la Fila 30961
df_steam_games['Id_Item'].fillna(200260.0, inplace=True)

In [27]:
# Convierte Columna 'Id_Item' en Tipo int
df_steam_games['Id_Item'] = df_steam_games['Id_Item'].astype('Int64')

---


### Se crea columna Año de Lanzamiento

In [28]:
# Extrae el año de la columna Fecha_Lanzamiento y crea columna Año_Lanzamiento
df_steam_games['Año_Lanzamiento'] = df_steam_games['Fecha_Lanzamiento'].str.extract(r'(\d{4})', expand=False)

In [38]:
# Convierte la columna Año_Lanzamiento a tipo de dato numérico
df_steam_games['Año_Lanzamiento'] = df_steam_games['Año_Lanzamiento'].astype('Int64')



---


### Desanidar las Listas


In [30]:
# Convierte las listas en la columna 'genres' en una cadena separada por comas
df_steam_games['Generos'] = df_steam_games['Generos'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Convierte las listas en la columna 'genres' en una cadena separada por comas
df_steam_games['Etiquetas'] = df_steam_games['Etiquetas'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Convierte las listas en la columna 'genres' en una cadena separada por comas
df_steam_games['Especificaciones'] = df_steam_games['Especificaciones'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [39]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30067 entries, 0 to 30066
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Empresa_Publicadora  30067 non-null  object 
 1   Generos              30067 non-null  object 
 2   Nombre_Contenido     30067 non-null  object 
 3   Titulo               30067 non-null  object 
 4   Url                  30067 non-null  object 
 5   Fecha_Lanzamiento    30067 non-null  object 
 6   Etiquetas            30067 non-null  object 
 7   Reviews_Contenido    30067 non-null  object 
 8   Especificaciones     30067 non-null  object 
 9   Precio               30067 non-null  float64
 10  Acceso_Temprano      30067 non-null  float64
 11  Id_Item              30067 non-null  Int64  
 12  Desarrollador        30067 non-null  object 
 13  Año_Lanzamiento      29966 non-null  Int64  
dtypes: Int64(2), float64(2), object(10)
memory usage: 3.3+ MB


### Rellenar columnas nulas con el valor 'sin datos'

In [32]:
# Crear una variable con las columnas a rellenar y modificar el DataFrame
columnas_a_llenar = ['Empresa_Publicadora', 'Generos', 'Reviews_Contenido', 'Especificaciones', 'Desarrollador']
df_steam_games[columnas_a_llenar] = df_steam_games[columnas_a_llenar].fillna('sin datos')


---


### Limpieza de caracteres especiales de las columnas

In [33]:
# Obteniendo los nombres de las columnas que se van limpiar
df_steam_games.columns

Index(['Empresa_Publicadora', 'Generos', 'Nombre_Contenido', 'Titulo', 'Url',
       'Fecha_Lanzamiento', 'Etiquetas', 'Reviews_Contenido',
       'Especificaciones', 'Precio', 'Acceso_Temprano', 'Id_Item',
       'Desarrollador', 'Año_Lanzamiento'],
      dtype='object')

In [34]:
# Crear una variable con las columnas para limpiar los caracteres
limpieza_filas = ['Empresa_Publicadora', 'Generos', 'Nombre_Contenido', 'Titulo', 'Etiquetas', 'Especificaciones', 'Desarrollador']

In [35]:
# Todos los strings en minúsculas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.astype(str).str.lower() if x.dtype == "object" else x)

# Reemplaza '-' por ' ' en todas las columnas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace('-', ' ') if x.dtype == "object" else x)

# Reemplaza '!' por '' en todas las columnas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace('!', '') if x.dtype == "object" else x)

# Reemplaza '¡' por '' en todas las columnas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace('¡', '') if x.dtype == "object" else x)

# Reemplaza '?' por '' en todas las columnas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace('?', '') if x.dtype == "object" else x)

# Reemplaza '¿' por '' en todas las columnas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace('¿', '') if x.dtype == "object" else x)

# Reemplaza '"' por '' en todas las columnas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace('"', '') if x.dtype == "object" else x)

# Reemplaza ''' por '' en todas las columnas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace("'", '') if x.dtype == "object" else x)

# Reemplaza '&' por '' en todas las columnas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace("&", '') if x.dtype == "object" else x)

# Reemplaza "''" por '' en todas las columnas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace("''", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace("_", '') if x.dtype == "object" else x)

# Reemplaza "." por '' en todas las columnas
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace(".", '') if x.dtype == "object" else x)

# Reemplaza "\s+" por ' ' espacios adicionales
df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace(r'\s+', ' ') if x.dtype == "object" else x)



<ipython-input-35-a7a3c23c8543>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace('?', '') if x.dtype == "object" else x)
<ipython-input-35-a7a3c23c8543>:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace(".", '') if x.dtype == "object" else x)
<ipython-input-35-a7a3c23c8543>:38: FutureWarning: The default value of regex will change from True to False in a future version.
  df_steam_games[limpieza_filas] = df_steam_games[limpieza_filas].apply(lambda x: x.str.replace(r'\s+', ' ') if x.dtype ==

In [36]:
# Restablecer el índice
df_steam_games = df_steam_games.reset_index(drop=True)

In [37]:
# Guardar el DataFrame comprimido en archivo tipo Parquet
df_steam_games.to_parquet('df_steam_games.parquet')

***
***

## ***3_ User Items:***




### Cargar JSON y guardar en una variable


In [ ]:
# Guardar la ruta en una variable
ruta = '/content/drive/MyDrive/Colab Notebooks/PI_1/Data/australian_users_items.json'

# Crear un lista vacia para almacenar las filas
fila = []

# Abrir el archivo JSON e iterar las Filas para rellenar la variable fila
with open(ruta, encoding="utf-8") as f:
    for line in f.readlines():
        fila.append(ast.literal_eval(line))

# Cargamos el archivo en un dataframe
user_items = pd.DataFrame(fila)

In [ ]:
# Dividir las celdas con su respectivo índice
df_items = user_items.explode('items')

In [ ]:
# Normaliza el DataFrame para agregar las columnas anidadas
df_items_nor = pd.json_normalize(df_items['items'])

In [ ]:
# Visualizacion del DataFrame
df_items_nor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   item_name         object 
 2   playtime_forever  float64
 3   playtime_2weeks   float64
dtypes: float64(2), object(2)
memory usage: 157.8+ MB


In [ ]:
# Visualizacion del DataFrame
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5170015 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      object
 1   items_count  int64 
 2   steam_id     object
 3   user_url     object
 4   items        object
dtypes: int64(1), object(4)
memory usage: 236.7+ MB


In [ ]:
# Se reinicia el indice para que coincidan
df_items = df_items.reset_index()

In [ ]:
# Elimina las ambas columnas
df_items = df_items.drop(['index', 'items'], axis=1)

In [ ]:
# Concatenan los dos DataFrame, para obtener todas las columnas
df_user_items = pd.concat([df_items, df_items_nor], axis=1)

In [ ]:
# Informacion general de como quedo el DataFrame
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


In [ ]:
# Muestra de las 5 primeras filas de DataFrame
df_user_items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0


In [ ]:
# Cambia el nombre de las columnas
df_user_items = df_user_items.rename(columns={
    'user_id': 'Id_Usuario',
    'items_count': 'Conteo_Items',
    'steam_id': 'Id_Steam',
    'user_url': 'Url_Usuario',
    'item_id': 'Id_Item',
    'item_name': 'Nombre_Juego',
    'playtime_forever': 'Tiempo_Jugado',
    'playtime_2weeks': 'Tiempo_Jugado_2_Semanas'
    })



---
### Valores Nulos:


In [ ]:
# Busacar filas con todos los Valores Nulos
filas_completas_nulas = df_user_items.isna().all(axis=1).sum()
print(f' Cantidad de Filas con todos Valores Nulos: {filas_completas_nulas}')

 Cantidad de Filas con todos Valores Nulos: 0


In [ ]:
# Cantidad de Valores Nulos por Columna
filas_con_nulos = df_user_items.isna().sum()
filas_con_nulos

Id_Usuario                     0
Conteo_Items                   0
Id_Steam                       0
Url_Usuario                    0
Id_Item                    16806
Nombre_Juego               16806
Tiempo_Jugado              16806
Tiempo_Jugado_2_Semanas    16806
dtype: int64

In [ ]:
# Buscar filas con al menos un valor nulo en columnas específicas
filas_nulas = df_user_items[['Id_Item', 'Nombre_Juego', 'Tiempo_Jugado', 'Tiempo_Jugado_2_Semanas']].isna().any(axis=1)
df_user_items[filas_nulas]

,Id_Usuario,Conteo_Items,Id_Steam,Url_Usuario,Id_Item,Nombre_Juego,Tiempo_Jugado,Tiempo_Jugado_2_Semanas
3733,Wackky,0,76561198039117046,http://steamcommunity.com/id/Wackky,NaN,NaN,NaN,NaN
3849,76561198079601835,0,76561198079601835,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN
6019,hellom8o,0,76561198117222320,http://steamcommunity.com/id/hellom8o,NaN,NaN,NaN,NaN
6523,starkillershadow553,0,76561198059648579,http://steamcommunity.com/id/starkillershadow553,NaN,NaN,NaN,NaN
7237,darkenkane,0,76561198058876001,http://steamcommunity.com/id/darkenkane,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5169470,76561198316380182,0,76561198316380182,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN
5169471,76561198316970597,0,76561198316970597,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN
5169472,76561198318100691,0,76561198318100691,http://steamcommunity.com/profiles/76561198318...,NaN,NaN,NaN,NaN
5170006,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,NaN,NaN,NaN,NaN


In [ ]:
# Elimina los Valores nulos que no se pueden rellenar
df_user_items = df_user_items.dropna(axis=0)

In [ ]:
# Se reinicia el indice para que coincidan
df_user_items = df_user_items.reset_index(drop=True)



 *   El total de Valores Nulos que poseen las filas de las columnas: 'Id_Item', 'Nombre_Juego', 'Tiempo_Jugado', 'Tiempo_Jugado_2_Semanas', es de 16806 y comparten las columnas, por lo que no se tiene datos, solamente el Id_Usuario y no aportan informacion necesaria para rellenar los Valores Nulos con algun otro metodo. Por este motivo se procede a eliminarlos.






---



### Cambiar el tipo de variable

In [ ]:
# Modifica el tipo de variable
df_user_items['Id_Usuario'] = df_user_items['Id_Usuario'].astype(str)
df_user_items['Id_Steam'] = df_user_items['Id_Steam'].astype(str)
df_user_items['Url_Usuario'] = df_user_items['Url_Usuario'].astype(str)
df_user_items['Nombre_Juego'] = df_user_items['Nombre_Juego'].astype(str)
df_user_items['Id_Item'] = df_user_items['Id_Item'].astype('int')




---





### Limpieza de Caracteres

In [ ]:
# Muestra el nombre de las columnas para crear la variable para la modificacion
df_user_items.columns

Index(['Id_Usuario', 'Conteo_Items', 'Id_Steam', 'Url_Usuario', 'Id_Item',
       'Nombre_Juego', 'Tiempo_Jugado', 'Tiempo_Jugado_2_Semanas'],
      dtype='object')

In [ ]:
# Crea una variable para guardar las columnas para realizar la limpieza de caracteres
limpieza_filas = ['Conteo_Items', 'Id_Steam', 'Id_Item', 'Nombre_Juego', 'Tiempo_Jugado', 'Tiempo_Jugado_2_Semanas']

In [ ]:
# Todos los strings en minúsculas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.astype(str).str.lower() if x.dtype == "object" else x)

# Reemplaza '-' por ' ' en todas las columnas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace('-', ' ') if x.dtype == "object" else x)

# Reemplaza '!' por '' en todas las columnas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace('!', '') if x.dtype == "object" else x)

# Reemplaza '¡' por '' en todas las columnas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace('¡', '') if x.dtype == "object" else x)

# Reemplaza '?' por '' en todas las columnas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace('?', '') if x.dtype == "object" else x)

# Reemplaza '¿' por '' en todas las columnas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace('¿', '') if x.dtype == "object" else x)

# Reemplaza '"' por '' en todas las columnas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace('"', '') if x.dtype == "object" else x)

# Reemplaza ''' por '' en todas las columnas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace("'", '') if x.dtype == "object" else x)

# Reemplaza '&' por '' en todas las columnas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace("&", '') if x.dtype == "object" else x)

# Reemplaza "''" por '' en todas las columnas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace("''", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace("_", '') if x.dtype == "object" else x)

# Reemplaza "." por '' en todas las columnas
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace(".", '') if x.dtype == "object" else x)

# Reemplaza "\s+" por ' ' espacios adicionales
df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace(r'\s+', ' ') if x.dtype == "object" else x)



<ipython-input-52-12a1931cb222>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace('?', '') if x.dtype == "object" else x)
<ipython-input-52-12a1931cb222>:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace(".", '') if x.dtype == "object" else x)
<ipython-input-52-12a1931cb222>:38: FutureWarning: The default value of regex will change from True to False in a future version.
  df_user_items[limpieza_filas] = df_user_items[limpieza_filas].apply(lambda x: x.str.replace(r'\s+', ' ') if x.dtype == "obje

In [ ]:
# Visualizar las 5 primeras filas
df_user_items.head()

,Id_Usuario,Conteo_Items,Id_Steam,Url_Usuario,Id_Item,Nombre_Juego,Tiempo_Jugado,Tiempo_Jugado_2_Semanas
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,counter strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,team fortress classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,day of defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,deathmatch classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,half life: opposing force,0.0,0.0




---

### Guarda DataFrame en comprimido en tipo Parquet

In [ ]:
# Guardar el DataFrame comprimido en Parquet
df_user_items.to_parquet('df_user_items.parquet')



---



---

